<a href="https://colab.research.google.com/github/khadkechetan/information_extraction/blob/main/Docvqa/Docvqa_imperia_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Owner: **Chetan Khadke** (khadkechetan@gmail.com)  ✌



In [ ]:
!pip install git+https://github.com/huggingface/transformers.git@2ef774211733f0acf8d3415f9284c49ef219e991
!pip install torch
!pip install datasets
!pip install pillow
!pip install sentencepiece  #  naver-clova-ix/donut-base-finetuned-docvqa based model

# Load Dataset

funsd dataset for example 😀

To import own data please assign followings.
- words into **words** 
- bounding box into **boxes**
- image into **image**


In [32]:
from datasets import load_dataset
from PIL import Image

dataset = load_dataset("nielsr/funsd", split="train")
example = dataset[0]
question = "What are the Manager comments??"
words = example["words"]
boxes = example["bboxes"]

In [33]:
image = Image.open(example["image_path"])

# Pipeline for **DOCVQA**

In [34]:
from transformers import pipeline

nlp = pipeline(
    "document-question-answering",
    model="impira/layoutlm-invoices",
    framework = "pt",
    # device=0  # use this for GPU
)
# model="naver-clova-ix/donut-base-finetuned-docvqa"
# model="impira/layoutlm-document-qa"

Some weights of the model checkpoint at impira/layoutlm-invoices were not used when initializing LayoutLMForQuestionAnswering: ['token_classifier_head.bias', 'token_classifier_head.weight']
- This IS expected if you are initializing LayoutLMForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LayoutLMForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Create batches of words and bboxes.

In [35]:
words_bbox = []
for word, bbox in zip(words,boxes):
    words_bbox.append([word,bbox])

In [ ]:
words_bbox

In [37]:
predication = nlp(
    image,
    "What is the advised solution?",
    word_boxes = words_bbox
)

In [38]:
predication

{'score': 0.019657475873827934,
 'answer': 'Discontinue coal retention analyses on licensee submitted product samples',
 'start': 36,
 'end': 44}

# Without Using the pipeline

In [39]:
from transformers import AutoProcessor, LayoutLMForQuestionAnswering, AutoTokenizer

import torch

tokenizer = AutoTokenizer.from_pretrained("impira/layoutlm-document-qa", add_prefix_space=True)
model = LayoutLMForQuestionAnswering.from_pretrained("impira/layoutlm-invoices")



encoding = tokenizer(
    question.split(), words, is_split_into_words=True, return_token_type_ids=True, return_tensors="pt"
)
bbox = []
for i, s, w in zip(encoding.input_ids[0], encoding.sequence_ids(0), encoding.word_ids(0)):
    if s == 1:
        bbox.append(boxes[w])
    elif i == tokenizer.sep_token_id:
        bbox.append([1000] * 4)
    else:
        bbox.append([0] * 4)
encoding["bbox"] = torch.tensor([bbox])

word_ids = encoding.word_ids(0)
outputs = model(**encoding)
loss = outputs.loss
start_scores = outputs.start_logits
end_scores = outputs.end_logits
start, end = word_ids[start_scores.argmax(-1)], word_ids[end_scores.argmax(-1)]
print("-----------------------------------")
print(" ".join(words[start : end + 1]))

Some weights of the model checkpoint at impira/layoutlm-invoices were not used when initializing LayoutLMForQuestionAnswering: ['token_classifier_head.bias', 'token_classifier_head.weight']
- This IS expected if you are initializing LayoutLMForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LayoutLMForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


-----------------------------------
Manager, please contact suggester and forward comments to the Quality Council.
